In [ ]:
import os

import cv2
import math
import numpy as np
import moviepy 
import torch

from matplotlib import pyplot as plt



from ref_alphaction.config import cfg
from ref_alphaction.modeling.detector import build_detection_model
from ref_alphaction.utils.checkpoint import ActionCheckpointer
from ref_alphaction.utils.comm import get_world_size



from tqdm import tqdm

In [ ]:
model_name = 'VMAEv2'


person_threshold = 0.6 # confidence threshold on actor. 0.6 is the defualt
sampling_rate = 3 # sampling rate: 4 is the defualt
top_k = 5 # number of actions per person
video_path = '../input_dir/markt2_fight.mp4'

slice_height = 800
slice_width = 1000
overlap_ratio = 0.1

starting_frame_index = 100
length_input = 200

exp_dict = {'model_name': model_name,
            'model_params': {'person_threshold': person_threshold, 
                             'sampling_rate': sampling_rate},
            'orig_post_processing':{'top_k': top_k},
            'aggregation': {'method': {}, 
                            'params': {}},
            'video_path': video_path,
            'slicing_params': {'slice_height': slice_height, 
                               'slice_width': slice_width, 
                               'overlap_ratio':overlap_ratio},
            'video_params': {'st_frame_index': starting_frame_index, 
                             'length_input':length_input
                             }
           }


In [ ]:
if model_name == 'VMAEv2':
    config_file = '../config_files/VMAEv2-ViTB-16x4.yaml'
if model_name == 'VMAE':
    config_file = '../config_files/VMAE-ViTB-16x4.yaml'


In [ ]:
cfg.merge_from_file(config_file)

In [ ]:
# change model weight path
if model_name == 'VMAEv2':
    cfg.merge_from_list(["MODEL.WEIGHT", "../checkpoints/VMAEv2_ViTB_16x4.pth"])
if model_name == 'VMAE':
    cfg.merge_from_list(["MODEL.WEIGHT", "../checkpoints/VMAE_ViTB_16x4.pth"])

# change output dir
cfg.merge_from_list(["OUTPUT_DIR", "../output_dir/"])

# change person threshold
cfg.merge_from_list(["MODEL.STM.PERSON_THRESHOLD", person_threshold])

# change sampling rate
cfg.merge_from_list(["DATA.SAMPLING_RATE", sampling_rate])

# change path for data_dir
cfg.merge_from_list(["DATA.PATH_TO_DATA_DIR", "/work/ava"])

# folder name of annotations
cfg.merge_from_list(["AVA.ANNOTATION_DIR", "annotations/"])

# file name of  frame_lists
cfg.merge_from_list(["AVA.TRAIN_LISTS", ['sample.csv']])
cfg.merge_from_list(["AVA.TEST_LISTS", ['sample.csv']])

# file name of predicted_bboxes
cfg.merge_from_list(["AVA.TRAIN_GT_BOX_LISTS", ['ava_sample_predicted_boxes.csv']])
cfg.merge_from_list(["AVA.TEST_GT_BOX_LISTS", ['ava_sample_predicted_boxes.csv']])

# file name of exlusions
cfg.merge_from_list(["AVA.EXCLUSION_FILE", 'ava_sample_train_excluded_timestamps_v2.2.csv'])

# number of batches in test scenario
cfg.merge_from_list(["TEST.VIDEOS_PER_BATCH", 1])

# number of workers
cfg.merge_from_list(["DATALOADER.NUM_WORKERS", 1])


In [ ]:
cfg.ViT.USE_CHECKPOINT

In [ ]:
cfg.merge_from_list(["ViT.USE_CHECKPOINT", False])

In [ ]:
cfg.ViT.USE_CHECKPOINT

In [ ]:
debug = True
if debug:
    # The shape of model input should be divisible into this. Otherwise, padding 0 to left and bottum. 
    print("cfg.DATALOADER.SIZE_DIVISIBILITY: ", cfg.DATALOADER.SIZE_DIVISIBILITY)
    
    # Sampling rate in constructing the clips.
    self_sample_rate =  cfg.DATA.SAMPLING_RATE
    print("cfg.DATA.SAMPLING_RATE: ", cfg.DATA.SAMPLING_RATE)
    
    # Length of clip
    self_video_length = cfg.DATA.NUM_FRAMES
    print("cfg.DATA.NUM_FRAMES: ", cfg.DATA.NUM_FRAMES)
    
    # Length of sequence frames from which a clip is constructed.
    self_seq_len = self_video_length * self_sample_rate
    print("self_seq_len: ", self_seq_len)
    
    self_num_classes = cfg.MODEL.STM.ACTION_CLASSES
    print("cfg.MODEL.STM.ACTION_CLASSES: ", self_num_classes)
    
    # Augmentation params.
    self_data_mean = cfg.DATA.MEAN
    self_data_std = cfg.DATA.STD
    self_use_bgr = cfg.AVA.BGR
    print("Augmentation params: ", self_data_mean, self_data_std, self_use_bgr)
    
    self_jitter_min_scale = cfg.DATA.TEST_MIN_SCALES
    self_jitter_max_scale = cfg.DATA.TEST_MAX_SCALE
    self_test_force_flip = cfg.AVA.TEST_FORCE_FLIP

    print("scale and flip params", self_jitter_min_scale, self_jitter_max_scale, self_test_force_flip)

In [ ]:
model = build_detection_model(cfg)

In [ ]:
checkpointer = ActionCheckpointer(cfg, model, save_dir="../output_dir/")
checkpointer.load(cfg.MODEL.WEIGHT)

In [ ]:
slow_video = torch.randn(1, 3, 16, 256, 320).to('cuda')


model = model.eval().to('cuda')

In [ ]:
model(slow_video)# NOTE THAT THE INPUT TO THIS VERISIN IS A TENSOR

In [ ]:
scripted_model = torch.jit.script(model)

In [ ]:
def foo(x, y):
    return torch.zeros_like(x) + y[1]

sc_fo = torch.jit.script(foo)

print(torch.jit.script(foo, (torch.rand(4, 4), [10,10])).code)

In [ ]:
cfg.ViT.USE_CHECKPOINT

In [ ]:
sc_fo(torch.rand(4, 4), [10,10])

In [ ]:
interpol = Interpolate()

In [ ]:
torch.jit.script(Interpolate())

In [2]:
import torch
from typing import Tuple


class Interpolate(torch.jit.ScriptModule):
    __constants__ = ["scale_factor", "mode", "align_corners"]

    def __init__(self, scale_factor=(15.0,15.0) , mode="bicubic", align_corners=None):
        super(Interpolate, self).__init__()
        self.scale_factor = scale_factor
        self.mode = mode
        self.align_corners = align_corners

    @torch.jit.script_method
    def forward(self, X):
        return nn.functional.interpolate(X, scale_factor=self.scale_factor,
                                         mode=self.mode, align_corners=self.align_corners)
    
def interpolate_pos_embed_online(
        pos_embed, orig_size: Tuple[int], new_size: Tuple[int], num_extra_tokens: int
):
    extra_tokens = pos_embed[:, :num_extra_tokens]
    pos_tokens = pos_embed[:, num_extra_tokens:]
    embedding_size = pos_tokens.shape[-1]
    pos_tokens = pos_tokens.reshape(
        -1, orig_size[0], orig_size[1], embedding_size
    ).permute(0, 3, 1, 2)
    pos_tokens = torch.nn.functional.interpolate(
        pos_tokens, size=new_size, mode="bicubic", align_corners=False,
    )
    pos_tokens = pos_tokens.permute(0, 2, 3, 1).flatten(1, 2)
    new_pos_embed = torch.cat((extra_tokens, pos_tokens), dim=1)
    return new_pos_embed

# Example input
pos_embed = torch.randn(3, 100, 20)  # Assuming 3 batches, 100 tokens, and 20 embedding dimensions
orig_size = (10, 10)  # Original size of positional embeddings
new_size = (15, 15)   # New size to interpolate to
num_extra_tokens = 0  # Number of extra tokens at the beginning of the positional embeddings

# Test the function
new_pos_embed = interpolate_pos_embed_online(pos_embed, orig_size, new_size, num_extra_tokens)
print("Shape of new positional embeddings:", new_pos_embed.shape)

Shape of new positional embeddings: torch.Size([3, 225, 20])


In [7]:
Interpolate(pos_embed)

TypeError: 
'torch.Tensor' object in attribute 'Interpolate.scale_factor' is not a valid constant.
Valid constants are:
1. a nn.ModuleList
2. a value of type {bool, float, int, str, NoneType, torch.device, torch.layout, torch.dtype}
3. a list or tuple of (2)


In [3]:
m = torch.jit.script(interpolate_pos_embed_online, (orig_size, new_size, num_extra_tokens))

/usr/local/lib/python3.8/dist-packages/torch/jit/_script.py:1243: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


RuntimeError: 
Tuple index out of range. Tuple is length 1 and index is 1:
  File "/tmp/ipykernel_3803/2346981515.py", line 26
    embedding_size = pos_tokens.shape[-1]
    pos_tokens = pos_tokens.reshape(
        -1, orig_size[0], orig_size[1], embedding_size
                          ~~~~~~~~~~~~ <--- HERE
    ).permute(0, 3, 1, 2)
    pos_tokens = torch.nn.functional.interpolate(


In [ ]:
def interpolate_pos_embed_online(
        pos_embed, orig_size: Tuple[int], new_size: Tuple[int], num_extra_tokens: int
):
    extra_tokens = pos_embed[:, :num_extra_tokens]
    pos_tokens = pos_embed[:, num_extra_tokens:]
    embedding_size = pos_tokens.shape[-1]
    pos_tokens = pos_tokens.reshape(
        -1, orig_size[0], orig_size[1], embedding_size
    ).permute(0, 3, 1, 2)
    pos_tokens = torch.nn.functional.interpolate(
        pos_tokens, size=new_size, mode="bicubic", align_corners=False,
    )
    pos_tokens = pos_tokens.permute(0, 2, 3, 1).flatten(1, 2)
    new_pos_embed = torch.cat((extra_tokens, pos_tokens), dim=1)
    return new_pos_embed

## INTERPOLATE POSE EMBED ONLINE

In [116]:

import torch

class MyInterpolate_Pose_Embed_Online(torch.nn.Module):
    def __init__(self, orig_size, new_size, num_extra_tokens):
        super(MyInterpolate_Pose_Embed_Online, self).__init__()
        self.orig_size = orig_size
        self.new_size = new_size
        self.num_extra_tokens = num_extra_tokens
    def forward(self, pos_embed):
        extra_tokens = pos_embed[:, :self.num_extra_tokens]
        pos_tokens = pos_embed[:, self.num_extra_tokens:]
        embedding_size = pos_tokens.shape[-1]
        pos_tokens = pos_tokens.reshape(
            -1, self.orig_size[0], self.orig_size[1], embedding_size).permute(0, 3, 1, 2)
        print(pos_tokens.shape)
        pos_tokens = torch.nn.functional.interpolate(
        pos_tokens, size=self.new_size, mode="bilinear", align_corners=False, )
        pos_tokens = pos_tokens.permute(0, 2, 3, 1).flatten(1, 2)
        new_pos_embed = torch.cat((extra_tokens, pos_tokens), dim=1)
        return new_pos_embed
    
    

In [117]:
pos_embed = torch.randn(3, 100, 20)  # Assuming 3 batches, 100 tokens, and 20 embedding dimensions
orig_size = (10, 10)  # Original size of positional embeddings
new_size = (15, 15)   # New size to interpolate to
num_extra_tokens = 0  # Number of extra tokens at the beginning of the positional embeddings



In [118]:
my_pose = MyInterpolate_Pose_Embed_Online(orig_size, new_size, num_extra_tokens)

In [119]:
my_pose(pos_embed)

torch.Size([3, 20, 10, 10])


tensor([[[ 1.6927, -1.0217,  0.8121,  ..., -1.2881,  0.1307,  0.7297],
         [ 1.6738,  0.2628,  0.7831,  ..., -0.7399, -0.6807, -0.0680],
         [ 1.2520,  1.3601,  0.6529,  ..., -0.3442, -1.1591, -0.6246],
         ...,
         [ 1.7018, -0.3035,  0.6705,  ..., -0.7306,  0.9061,  0.0886],
         [ 1.2231,  0.0390,  0.6591,  ..., -0.6369, -0.0131,  0.0208],
         [ 0.2063,  0.7071,  0.4055,  ..., -0.4308, -1.2560, -0.3108]],

        [[-0.0799, -0.8515,  0.3789,  ...,  0.2424, -0.3138,  0.5497],
         [-0.2181, -0.3297, -0.3611,  ..., -0.0637,  0.6720, -0.7122],
         [-0.6433, -0.0598, -1.0004,  ..., -0.2314,  1.6398, -1.5495],
         ...,
         [-0.4789, -0.7532, -0.7629,  ...,  0.4329,  1.0429, -0.1593],
         [-0.7032, -0.6635, -0.9407,  ...,  0.2803,  1.0328,  0.0843],
         [-0.8350, -0.5540, -0.9626,  ...,  0.3472,  0.7101,  0.5344]],

        [[ 1.1857,  1.1835,  0.4705,  ..., -1.0137, -1.4482,  0.1568],
         [-0.3859,  1.4280, -0.6051,  ..., -0

In [120]:
my_pose_scipted = torch.jit.script(my_pose)

In [121]:
my_pose_scipted(pos_embed)

[3, 20, 10, 10]


tensor([[[ 1.6927, -1.0217,  0.8121,  ..., -1.2881,  0.1307,  0.7297],
         [ 1.6738,  0.2628,  0.7831,  ..., -0.7399, -0.6807, -0.0680],
         [ 1.2520,  1.3601,  0.6529,  ..., -0.3442, -1.1591, -0.6246],
         ...,
         [ 1.7018, -0.3035,  0.6705,  ..., -0.7306,  0.9061,  0.0886],
         [ 1.2231,  0.0390,  0.6591,  ..., -0.6369, -0.0131,  0.0208],
         [ 0.2063,  0.7071,  0.4055,  ..., -0.4308, -1.2560, -0.3108]],

        [[-0.0799, -0.8515,  0.3789,  ...,  0.2424, -0.3138,  0.5497],
         [-0.2181, -0.3297, -0.3611,  ..., -0.0637,  0.6720, -0.7122],
         [-0.6433, -0.0598, -1.0004,  ..., -0.2314,  1.6398, -1.5495],
         ...,
         [-0.4789, -0.7532, -0.7629,  ...,  0.4329,  1.0429, -0.1593],
         [-0.7032, -0.6635, -0.9407,  ...,  0.2803,  1.0328,  0.0843],
         [-0.8350, -0.5540, -0.9626,  ...,  0.3472,  0.7101,  0.5344]],

        [[ 1.1857,  1.1835,  0.4705,  ..., -1.0137, -1.4482,  0.1568],
         [-0.3859,  1.4280, -0.6051,  ..., -0

In [122]:
torch.onnx.export(my_pose_scipted, 
                  args = pos_embed, 
                  f="pose_embeding.onnx", 
                  input_names=["pose_embed"], 
                  output_names=["output"],
                  opset_version=11,
                  verbose=False
                  #dynamic_axes={"slow_video": {0: "batch_size"}, "whwh": {0: "batch_size"}, "output": {0: "batch_size"}}
                 )

[3, 20, 10, 10]


In [123]:
import onnx
from onnx import helper
from onnx import numpy_helper

# Load the ONNX model
pose_embeding_onnx = onnx.load("pose_embeding.onnx")


In [124]:
print(onnx.helper.printable_graph(pose_embeding_onnx.graph))

graph torch_jit (
  %pose_embed[FLOAT, 3x100x20]
) {
  %/Constant_output_0 = Constant[value = <Tensor>]()
  %/Constant_1_output_0 = Constant[value = <Tensor>]()
  %/Constant_2_output_0 = Constant[value = <Tensor>]()
  %/Constant_3_output_0 = Constant[value = <Tensor>]()
  %/Slice_output_0 = Slice(%pose_embed, %/Constant_1_output_0, %/Constant_2_output_0, %/Constant_output_0, %/Constant_3_output_0)
  %/Constant_4_output_0 = Constant[value = <Tensor>]()
  %/Reshape_output_0 = Reshape(%pose_embed, %/Constant_4_output_0)
  %/Transpose_output_0 = Transpose[perm = [0, 3, 1, 2]](%/Reshape_output_0)
  %onnx::Slice_17 = Shape(%/Transpose_output_0)
  %onnx::Slice_18 = Constant[value = <Tensor>]()
  %onnx::Slice_19 = Constant[value = <Tensor>]()
  %onnx::Slice_20 = Constant[value = <Tensor>]()
  %onnx::Concat_21 = Slice(%onnx::Slice_17, %onnx::Slice_19, %onnx::Slice_20, %onnx::Slice_18)
  %onnx::Concat_46 = Constant[value = <Tensor>]()
  %onnx::Resize_23 = Concat[axis = 0](%onnx::Concat_21, %onnx

In [125]:
import onnxruntime as ort
import numpy as np

ort_session = ort.InferenceSession("pose_embeding.onnx")
pose_embed_np = pos_embed.numpy().astype(np.float32)

onnx.checker.check_model("pose_embeding.onnx")


In [126]:
# Run inference
outputs = ort_session.run(["output"], {"pose_embed": pose_embed_np})


In [14]:
pos_embed.numpy()

array([[[-0.64137936,  0.02309975, -0.23200642, ..., -0.32407302,
          1.519191  ,  0.5005934 ],
        [ 0.00505867,  0.78447485, -0.09199175, ...,  0.89557576,
         -1.5344915 , -1.5433446 ],
        [-1.0785986 ,  0.5127451 ,  0.17516923, ..., -2.270945  ,
         -0.14369453, -0.7394889 ],
        ...,
        [-1.3477877 , -0.42172876,  0.02565104, ...,  0.748587  ,
         -0.3982138 ,  0.7097116 ],
        [-1.8790638 ,  0.64939225, -0.9743109 , ...,  0.34152257,
          1.6852396 ,  1.177382  ],
        [ 0.542683  , -1.2692044 ,  0.42710724, ..., -0.8933482 ,
         -0.8667681 , -1.0421079 ]],

       [[ 0.40192738, -1.4155567 , -2.0919912 , ..., -1.4260367 ,
         -1.3999703 , -0.4659327 ],
        [-1.0190424 , -0.27929085, -2.0398881 , ..., -0.8584975 ,
          0.24301495, -0.58877426],
        [ 1.532979  ,  1.101683  ,  0.61486924, ...,  1.5265759 ,
          0.629919  ,  0.538811  ],
        ...,
        [-0.7861958 , -1.4978276 ,  0.9127483 , ...,  

In [52]:
import torch


class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv = torch.nn.Conv3d(5, 1, 3, padding=1, bias=False)

    def forward(self, x):
        new_x = self.conv(x)
        up_x = torch.nn.functional.interpolate(
            new_x, scale_factor=(2,2,2), mode="bicubic", align_corners=True)
        return up_x


inp_5 = torch.randn(1, 5, 5, 5, 5)
inp_10 = torch.randn(1, 5, 10, 10, 10)
inp_15 = torch.randn(1, 5, 15, 15, 15)

model = Model()
model.eval()
trace = torch.jit.trace(model, inp_10)
trace.save("trace.pth")

result_model_5 = model(inp_5)
result_model_10 = model(inp_10)
result_model_15 = model(inp_15)
print("Shape  5, {} ||| {}".format(result_model_5.shape, result_model_5.shape))
print("Shape 10, {} ||| {}".format(result_model_10.shape, result_model_10.shape))
print("Shape 15, {} ||| {}".format(result_model_15.shape, result_model_15.shape))
t_model = torch.jit.load("trace.pth")
result_t_model_5 = t_model(inp_5)
result_t_model_10 = t_model(inp_10)
result_t_model_15 = t_model(inp_15)

print("Shape  5, {} ||| {}".format(result_model_5.shape, result_t_model_5.shape))
print("Shape 10, {} ||| {}".format(result_model_10.shape, result_t_model_10.shape))
print("Shape 15, {} ||| {}".format(result_model_15.shape, result_t_model_15.shape))
torch.allclose(result_model_5, result_t_model_5)
torch.allclose(result_model_10, result_t_model_10)
torch.allclose(result_model_15, result_t_model_15)

NotImplementedError: Input Error: Only 3D, 4D and 5D input Tensors supported (got 5D) for the modes: nearest | linear | bilinear | bicubic | trilinear | area | nearest-exact (got bicubic)

In [19]:
import torch
from torch import nn


class Model(torch.nn.Module):
    def __init__(self, scale_factor):
        super(Model, self).__init__()
        self.scale_factor = scale_factor
    def forward(self, x):
        xp = nn.functional.interpolate(
            x,
            scale_factor=(self.scale_factor),
            mode="bicubic",
        )
        return xp



model = Model((10,10))
model.eval()
x = torch.randn(1, 3, 224, 224)
y = model(x)
print(y.shape)

# This will fail
torch.jit.trace(model, x)

torch.Size([1, 3, 2240, 2240])


Model(original_name=Model)

In [26]:
torch.jit.script(Model(torch.tensor((10,10))))

RecursiveScriptModule(original_name=Model)

In [38]:
class Modelv2(torch.nn.Module):
    def __init__(self):
        super(Modelv2, self).__init__()
    def forward(self, x, scale_factor):
        xp = nn.functional.interpolate(
            x,
            scale_factor=(scale_factor),
            mode="bicubic",
        )
        return xp



In [39]:
scale_factor = (10, 10)
modelv2 = Modelv2()

In [40]:
modelv2(x, scale_factor)

tensor([[[[ 0.2163,  0.2205,  0.2179,  ..., -2.2976, -2.2993, -2.2966],
          [ 0.2241,  0.2285,  0.2258,  ..., -2.3270, -2.3287, -2.3259],
          [ 0.2193,  0.2236,  0.2209,  ..., -2.3089, -2.3105, -2.3078],
          ...,
          [ 2.2916,  2.3073,  2.2976,  ..., -0.2974, -0.2997, -0.2960],
          [ 2.2995,  2.3152,  2.3055,  ..., -0.2991, -0.3014, -0.2977],
          [ 2.2868,  2.3024,  2.2928,  ..., -0.2963, -0.2986, -0.2949]],

         [[-0.7826, -0.7892, -0.7851,  ..., -0.5792, -0.5839, -0.5763],
          [-0.7868, -0.7935, -0.7894,  ..., -0.5793, -0.5840, -0.5763],
          [-0.7842, -0.7909, -0.7868,  ..., -0.5792, -0.5839, -0.5763],
          ...,
          [ 0.4657,  0.4742,  0.4689,  ...,  1.0051,  1.0062,  1.0044],
          [ 0.4671,  0.4756,  0.4704,  ...,  1.0064,  1.0074,  1.0057],
          [ 0.4648,  0.4732,  0.4680,  ...,  1.0043,  1.0054,  1.0036]],

         [[-1.9678, -1.9872, -1.9752,  ..., -0.2263, -0.2298, -0.2241],
          [-1.9812, -2.0006, -

In [45]:
scripted_modelv2 = torch.jit.script(Modelv2())

In [46]:
scripted_modelv2(x, torch.tensor(scale_factor))

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/tmp/ipykernel_3803/558728084.py", line 5, in forward
    def forward(self, x, scale_factor):
        xp = nn.functional.interpolate(
             ~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
            x,
            scale_factor=(scale_factor),
RuntimeError: Cannot input a tensor of dimension other than 0 as a scalar argument


In [47]:
scripted_modelv2(x, scale_factor)

RuntimeError: forward() Expected a value of type 'Tensor (inferred)' for argument 'scale_factor' but instead found type 'tuple'.
Inferred 'scale_factor' to be of type 'Tensor' because it was not annotated with an explicit type.
Position: 2
Value: (10, 10)
Declaration: forward(__torch__.___torch_mangle_21.Modelv2 self, Tensor x, Tensor scale_factor) -> Tensor
Cast error details: Unable to cast (10, 10) to Tensor

In [56]:
class Modelv3(torch.jit.ScriptModule):
    def __init__(self):
        super(Modelv3, self).__init__()
        
    @torch.jit.script_method 
    def forward(self, x, scale_factor):
        
        xp = nn.functional.interpolate(
            x,
            scale_factor=(scale_factor),
            mode="bicubic",
        )
        return xp



In [57]:
scripted_modelv3 = Modelv3()

scripted_modelv3(x, scale_factor)

In [58]:
scripted_modelv3(x, scale_factor)

RuntimeError: forward() Expected a value of type 'Tensor (inferred)' for argument 'scale_factor' but instead found type 'tuple'.
Inferred 'scale_factor' to be of type 'Tensor' because it was not annotated with an explicit type.
Position: 2
Value: (10, 10)
Declaration: forward(__torch__.___torch_mangle_23.Modelv3 self, Tensor x, Tensor scale_factor) -> Tensor
Cast error details: Unable to cast (10, 10) to Tensor

In [59]:
scripted_modelv3(x, torch.tensor(scale_factor))

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/tmp/ipykernel_3803/4183376326.py", line 8, in forward
    def forward(self, x, scale_factor):
        
        xp = nn.functional.interpolate(
             ~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
            x,
            scale_factor=(scale_factor),
RuntimeError: Cannot input a tensor of dimension other than 0 as a scalar argument


In [60]:
class Modelv4(torch.jit.ScriptModule):
    def __init__(self):
        super(Modelv4, self).__init__()
        
    @torch.jit.script_method 
    def forward(self, x, scale_factor):
        
        scale_factor_list = scale_factor.tolist()
        xp = nn.functional.interpolate(
            x,
            scale_factor=(scale_factor_list),
            mode="bicubic",
        )
        return xp


In [61]:
scripted_modelv4 = Modelv4()

RuntimeError: 
Expected type hint for result of tolist():
  File "/tmp/ipykernel_3803/2478107352.py", line 8
    def forward(self, x, scale_factor):
        
        scale_factor_list = scale_factor.tolist()
                            ~~~~~~~~~~~~~~~~~~~ <--- HERE
        xp = nn.functional.interpolate(
            x,
